In [1]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip -q install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
!pip -q install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
!pip -q install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
!pip -q install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
!pip -q install torch-geometric
!pip -q install torch-geometric-temporal

import random
import logging
import pickle
import os 
from collections import defaultdict
import copy
import math
import time 
import numpy as np
np.set_printoptions(threshold=np.inf)

import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv

import dill

class Namespace(object):
    '''
    helps referencing object in a dictionary as dict.key instead of dict['key']
    '''
    def __init__(self, adict):
        self.__dict__.update(adict)



with open('../input/data-preprocessing-output/data.pkl', 'rb') as f:
    train_data = dill.load(f)
    valid_data = dill.load(f)
    test_data = dill.load(f)
    diffusion_graph = dill.load(f)
    
    

1.7.0
10.2
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
opt = Namespace({})
opt.epoch=50

opt.d_model=64
opt.n_warmup_steps=1000
opt.dropout=0.1
opt.log=None
#opt.save_path=root_path + "checkpoint/DiffusionPrediction.pt"
opt.save_mode='best'
opt.network=False # use social network; need features or deepwalk embeddings as initial input
opt.pos_emb=True
opt.warmup=10 # warmup epochs
opt.notes=''
opt.d_word_vec = opt.d_model
opt.user_size = train_data.user_size


Constants = Namespace({})
Constants.PAD = 0

In [3]:
class TransformerBlock(nn.Module):
    def __init__(self, input_size, d_k=64, d_v=64, n_heads=2, is_layer_norm=True, attn_dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.n_heads = n_heads
        self.d_k = d_k if d_k is not None else input_size
        self.d_v = d_v if d_v is not None else input_size

        self.is_layer_norm = is_layer_norm
        if is_layer_norm:
            self.layer_morm = nn.LayerNorm(normalized_shape=input_size)  # 71

        # self.pos_encoding = PositionalEncoding(d_model=input_size, dropout=0.5)
        self.W_q = nn.Parameter(torch.Tensor(input_size, n_heads * d_k)) # 72, 8 * 64
        self.W_k = nn.Parameter(torch.Tensor(input_size, n_heads * d_k))
        self.W_v = nn.Parameter(torch.Tensor(input_size, n_heads * d_v))

        self.W_o = nn.Parameter(torch.Tensor(d_v*n_heads, input_size)) # 512 * 72
        self.linear1 = nn.Linear(input_size, input_size)
        self.linear2 = nn.Linear(input_size, input_size)

        self.dropout = nn.Dropout(attn_dropout)
        self.__init_weights__()
        print(self)

    def __init_weights__(self):
        init.xavier_normal_(self.W_q)
        init.xavier_normal_(self.W_k)
        init.xavier_normal_(self.W_v)
        init.xavier_normal_(self.W_o)

        init.xavier_normal_(self.linear1.weight)
        init.xavier_normal_(self.linear2.weight)

    def FFN(self, X):
        output = self.linear2(F.relu(self.linear1(X)))
        output = self.dropout(output)
        return output

    def scaled_dot_product_attention(self, Q, K, V, mask, episilon=1e-6):
        '''
        :param Q: (*, max_q_words, n_heads, input_size) # 128 * 438 * 64
        :param K: (*, max_k_words, n_heads, input_size)
        :param V: (*, max_v_words, n_heads, input_size)
        :param mask: (*, max_q_words) # 128 * 438
        :param episilon:
        :return:
        '''

        temperature = self.d_k ** 0.5 # d_k = 64 so temp is 8
        Q_K = torch.einsum("bqd,bkd->bqk", Q, K) / (temperature + episilon)  # 128 * 438 * 438

        if mask is not None:
            pad_mask = mask.unsqueeze(dim=-1).expand(-1, -1, K.size(1))  # 128 * 438 * 438
            mask = torch.triu(torch.ones(pad_mask.size()), diagonal=1).bool().cuda()
            mask_ = mask + pad_mask
            Q_K = Q_K.masked_fill(mask_, -2**32+1)

        Q_K_score = F.softmax(Q_K, dim=-1)  # (batch_size, max_q_words, max_k_words)
        Q_K_score = self.dropout(Q_K_score)

        V_att = Q_K_score.bmm(V)  # (*, max_q_words, input_size) # 128 * 438 * 64

        return V_att


    def multi_head_attention(self, Q, K, V, mask):
        '''
        :param Q:
        :param K:
        :param V:
        :param mask: (bsz, max_q_words)
        :return:
        '''
        bsz, q_len, _ = Q.size() #[16,438, 72]
        bsz, k_len, _ = K.size()
        bsz, v_len, _ = V.size()


        Q_ = Q.matmul(self.W_q).view(bsz, q_len, self.n_heads, self.d_k) 
        K_ = K.matmul(self.W_k).view(bsz, k_len, self.n_heads, self.d_k)
        V_ = V.matmul(self.W_v).view(bsz, v_len, self.n_heads, self.d_v)

        Q_ = Q_.permute(0, 2, 1, 3).contiguous().view(bsz*self.n_heads, q_len, self.d_k)
        K_ = K_.permute(0, 2, 1, 3).contiguous().view(bsz*self.n_heads, q_len, self.d_k)
        V_ = V_.permute(0, 2, 1, 3).contiguous().view(bsz*self.n_heads, q_len, self.d_v)

        if mask is not None:
            mask = mask.unsqueeze(dim=1).expand(-1, self.n_heads, -1)  # For head axis broadcasting.
            mask = mask.reshape(-1, mask.size(-1))


        V_att = self.scaled_dot_product_attention(Q_, K_, V_, mask)  #128 * 438 * 64
        V_att = V_att.view(bsz, self.n_heads, q_len, self.d_v) # 16 * 8 * 438 * 64
        V_att = V_att.permute(0, 2, 1, 3).contiguous().view(bsz, q_len, self.n_heads*self.d_v)  # 16 * 438 * 512

        output = self.dropout(V_att.matmul(self.W_o)) # (batch_size, max_q_words, input_size)  # #W_o 512 * 72  # 16 * 438 * 72
        return output


    def forward(self, Q, K, V, mask=None):
        '''
        :param Q: (batch_size, max_q_words, input_size) # 16 * 438 * 72 
        :param K: (batch_size, max_k_words, input_size)
        :param V: (batch_size, max_v_words, input_size)
        :return:  output: (batch_size, max_q_words, input_size)  same size as Q
        '''
 
        V_att = self.multi_head_attention(Q, K, V, mask)  # mask (16, 243)  #  # 16 * 438 * 72

        if self.is_layer_norm:
            X = self.layer_morm(Q + V_att)  # (batch_size, max_r_words, embedding_dim) # 16 * 438 * 72

            output = self.layer_morm(self.FFN(X) + X)    # 16 * 438 * 72
        else:
            X = Q + V_att  # 16 * 438 * 72
            output = self.FFN(X) + X   # 16 * 438 * 72

        return output

In [4]:
def get_previous_user_mask(seq, user_size):
    ''' Mask previous activated users.'''
    assert seq.dim() == 2  # 16 * 201
    prev_shape = (seq.size(0), seq.size(1), seq.size(1)) # 16, 201, 201
    seqs = seq.repeat(1, 1, seq.size(1)).view(seq.size(0), seq.size(1), seq.size(1)) # 16, 201, 201
    previous_mask = np.tril(np.ones(prev_shape)).astype('float32')
    previous_mask = torch.from_numpy(previous_mask)
    if seq.is_cuda:
        previous_mask = previous_mask.cuda()

    masked_seq = previous_mask * seqs.data.float() # 16, 201, 201


    # force the 0th dimension (Constants.PAD) to be masked
    PAD_tmp = torch.zeros(seq.size(0), seq.size(1), 1) # 16, 201, 1
    if seq.is_cuda:
        PAD_tmp = PAD_tmp.cuda()
    masked_seq = torch.cat([masked_seq, PAD_tmp], dim=2) # was [16, 201, 201]  becomes [16, 201, 202]
    
    ans_tmp = torch.zeros(seq.size(0), seq.size(1), user_size) # 16 201 12629
    if seq.is_cuda:
        ans_tmp = ans_tmp.cuda()
        

    masked_seq = ans_tmp.scatter_(2, masked_seq.long(), float('-inf'))
    masked_seq = Variable(masked_seq, requires_grad=False) # 16 201 12629
    return masked_seq

In [5]:

class Metrics(object):
    def __init__(self):
        super().__init__()
        self.PAD = 0

    def apk(self, actual, predicted, k=10):
        """
        Computes the average precision at k between two lists of items.
        actual :  A list of elements that are to be predicted (order doesn't matter)
        predicted :  A list of predicted elements (order does matter)
        k : int, optional  The maximum number of predicted elements
        """
        score = 0.0
        num_hits = 0.0

        for i, p in enumerate(predicted):
            if p in actual and p not in predicted[:i]:
                num_hits += 1.0
                score += num_hits / (i + 1.0)

        # if not actual:
        # 	return 0.0
        return score / min(len(actual), k)


    def compute_metric(self, y_prob, y_true, k_list=[10, 50, 100]):
        '''
            y_true: (#samples, )
            y_pred: (#samples, #users)
        '''
        scores_len = 0
        y_prob = np.array(y_prob)
        y_true = np.array(y_true)

        scores = {'hits@'+str(k):[] for k in k_list}
        scores.update({'map@'+str(k):[] for k in k_list})
        for p_, y_ in zip(y_prob, y_true):
            if y_ != self.PAD:
                scores_len += 1.0
                p_sort = p_.argsort()
                for k in k_list:
                    topk = p_sort[-k:][::-1]
                    scores['hits@' + str(k)].extend([1. if y_ in topk else 0.])
                    scores['map@'+str(k)].extend([self.apk([y_], topk, k)])

        scores = {k: np.mean(v) for k, v in scores.items()}
        return scores, scores_len
    
metric = Metrics()

In [6]:
import torch
from torch.nn import GRU
from torch_geometric.nn import GCNConv
from torch_geometric.nn import TopKPooling


In [7]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [8]:
class DyHGCN_H(nn.Module):

    def __init__(self, diffusion_graph, opt):
        super(DyHGCN_H, self).__init__()
        ntoken = opt.user_size # training 12629
        self.ninp = 64
        self.user_size = ntoken # training 12629
        self.pos_dim = 8
        self.__name__ = "DyHGCN_H"
        
        #
        self.num_of_nodes = ntoken
        self.in_channels = 64
        #
        self.embedding = nn.Embedding(ntoken, self.ninp, padding_idx=0) # 12629, 64
        self.ratio = self.ninp / self.num_of_nodes
        self.pooling_layer = TopKPooling( self.ninp, self.ratio)
        
        self.gnn1 = GCNConv(self.ninp, self.ninp,  bias = False)

        self.recurrent_layer1 = GRU(input_size = self.ninp,
                                   hidden_size = self.ninp,
                                   num_layers = 1)
        
        self.gnn2 = GCNConv(self.ninp, self.ninp, bias = False)
        self.recurrent_layer2 = GRU(input_size = self.ninp,
                           hidden_size = self.ninp,
                           num_layers = 1)
        self.diffusion_graph_list = diffusion_graph
        #------------------
        self.pos_embedding = nn.Embedding(1000, self.pos_dim)
        
        self.pos_encoder = PositionalEncoding(8, 0.1)

        self.decoder_attention = TransformerBlock(input_size=self.ninp + self.pos_dim, n_heads=8)  #72
        self.linear = nn.Linear(self.ninp + self.pos_dim, ntoken) # 72 * 12629


    def forward(self, input, input_timestamp):
        mask = (input == Constants.PAD)

        batch_size, max_len = input.size()  # 16, 201
        
        batch_t = torch.arange(max_len).expand((batch_size, max_len)).cuda()

        order_embed = self.pos_embedding(batch_t)   # 16, 201, 8
        order_embed = self.pos_encoder(order_embed) # 16, 201, 8

        batch_all = torch.arange(self.user_size).cuda()

        dyemb = torch.zeros(batch_size, max_len, self.ninp).cuda() # 16, 201, 64

        dynamic_node_emb_dict = dict()
        graph_embedding_list = list() 
        X_tilde_all = []
        for key in sorted(self.diffusion_graph_list.keys()):
            graph = self.diffusion_graph_list[key] 
            graph_edge_index = graph.edge_index.cuda()
            embeds = self.embedding(batch_all.cuda())
            
            graph_x_embeddings = embeds.cuda()  # 16, 12500, 64  # [12629, 64]
            X_tilde = self.pooling_layer(graph_x_embeddings, graph_edge_index) # [64, 64]
            X_tilde_all.append(X_tilde[0][None,:,:])

            graph_x_embeddings = nn.Dropout(0.3)(self.gnn1(graph_x_embeddings, graph_edge_index))  # [12629, 64]
            graph_x_embeddings = nn.Dropout(0.1)(self.gnn2(graph_x_embeddings, graph_edge_index))  # [12629, 64]



            graph_embedding_list.append(graph_x_embeddings)
            dynamic_node_emb_dict[key] = graph_x_embeddings
        
        W1 = self.gnn1.weight[None, :, :] # [1, 64, 64]
        #W2 = self.gnn2.weight[None, :, :] # [1, 64, 64]

        X_tilde_all = torch.cat(X_tilde_all, dim = 0) # [8, 64, 64]

        _, W1 = self.recurrent_layer1(X_tilde_all, W1) #[1, 64, 64] , [1, 64, 64] 
        #X_tilde_all, W2 = self.recurrent_layer1(X_tilde_all, W2) #[1, 64, 64] , [1, 64, 64] 

        self.gnn1.weight = torch.nn.Parameter(W1.squeeze())
        #self.gnn2.weight = torch.nn.Parameter(W2.squeeze())


        latest_timestamp = sorted(dynamic_node_emb_dict.keys())[-1]
        
        step_len = 5 

        for t in range(0, max_len, step_len):
            try:
                la_timestamp = torch.max(input_timestamp[:, t:t+step_len]).item()

                if la_timestamp < 1:
                    break 
                latest_timestamp = la_timestamp 
            except Exception:
                # print (input_timestamp[:, t:t+step_len])
                pass 

            his_timestamp = sorted(dynamic_node_emb_dict.keys())[-1]
            for x in sorted(dynamic_node_emb_dict.keys()):
                if x <= latest_timestamp:
                    his_timestamp = x
                    continue
                else:
                    break 
            

            graph_dynamic_embeddings = dynamic_node_emb_dict[his_timestamp]
            
            dyemb[:, t:t+step_len, :] = F.embedding(input[:, t:t+step_len].cuda(), graph_dynamic_embeddings.cuda())


        final_embed = torch.cat([dyemb, order_embed], dim=-1).cuda() # dynamic_node_emb
        
        
        att_out = self.decoder_attention(final_embed.cuda(), final_embed.cuda(), final_embed.cuda(), mask=mask.cuda())   # 16 * 438 * 72
        att_out = att_out.cuda()

        output = self.linear(att_out.cuda())  # (bsz, user_len, |U|) linear( 72 , 12629)  # 16 * 438 * 12629
        
        mask = get_previous_user_mask(input.cuda(), self.user_size)   #user_size 12629
        output = output.cuda() + mask.cuda()  # 16 * 438 * 12629
        return output.view(-1, output.size(-1)) #16 * 438 , 12629

In [9]:
model = DyHGCN_H(diffusion_graph, opt)  
loss_func = nn.CrossEntropyLoss(size_average=False, ignore_index=Constants.PAD)

params = model.parameters()
optimizerAdam = torch.optim.Adam(params, betas=(0.9, 0.98), eps=1e-09)

class ScheduledOptim(object): # A wrapper class for optimizer  for learning rate scheduling
    def __init__(self, optimizer, d_model, n_warmup_steps):
        self.optimizer = optimizer
        self.d_model = d_model
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0

    def step(self):  # "Step by the inner optimizer"
        self.optimizer.step()

    def zero_grad(self): # "Zero out the gradients by the inner optimizer"
        self.optimizer.zero_grad()

    def update_learning_rate(self):   #''' Learning rate scheduling per step '''
        self.n_current_steps += 1
        new_lr = np.power(self.d_model, -0.5) * np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

        for param_group in self.optimizer.param_groups:
            param_group['lr'] = new_lr
            
            
optimizer = ScheduledOptim(optimizerAdam, opt.d_model, opt.n_warmup_steps)

if torch.cuda.is_available():
    model = model.cuda()
    loss_func = loss_func.cuda()
    

validation_history = 0.0


TransformerBlock(
  (layer_morm): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
  (linear1): Linear(in_features=72, out_features=72, bias=True)
  (linear2): Linear(in_features=72, out_features=72, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [10]:
def doit(data_partition, label):
    ''' Epoch operation in evaluation phase '''
    model.eval()

    k_list=[10, 50, 100]
    scores_checker = {}
    for k in k_list:
        scores_checker['hits@' + str(k)] = 0
        scores_checker['map@' + str(k)] = 0

    n_total_words_checker = .01
    
    for i, batch in enumerate(data_partition): 
            print("{} batch ".format(label), i)
            # prepare data
            tgt, tgt_timestamp = batch  # tgt = 16, 209

            y_gold = tgt[:, 1:].contiguous().view(-1).detach().cpu().numpy() # 16*208
            input_tgt = tgt[:, :-1]
            input_timestamp = tgt_timestamp[:, :-1] 
            # forward
            pred = model(input_tgt, input_timestamp)
            y_pred = pred.detach().cpu().numpy()


            scores_batch, scores_len = metric.compute_metric(y_pred, y_gold, k_list)
            n_total_words_checker += scores_len
            for k in k_list:
                scores_checker['hits@' + str(k)] += scores_batch['hits@' + str(k)] * scores_len
                scores_checker['map@' + str(k)] += scores_batch['map@' + str(k)] * scores_len
            break;

    for k in k_list:
        scores_checker['hits@' + str(k)] = scores_checker['hits@' + str(k)] / n_total_words_checker
        scores_checker['map@' + str(k)] = scores_checker['map@' + str(k)] / n_total_words_checker

    for metric_ in scores_checker.keys():
        print(metric_ + ' ' + str(scores_checker[metric_]))

    #test_scores = test_epoch(model, test_data, diffusion_graph)
    #for metric_ in test_scores.keys():
    #                print(metric_ + ' ' + str(test_scores[metric_]))
    model.train()
    return scores_checker

In [11]:
start_again = False
if start_again:
    scores_train = []
    scores_valid = []
    scores_test = []

model.train()
for _ in range(100):
    total_loss = 0.0
    n_total_words = 0.0
    n_total_correct = 0.0
    batch_num = 0.0

    for i, batch in enumerate(train_data):
        i = i 
        batch = batch
        # prepare data
        tgt, tgt_timestamp = (item.cuda() for item in batch)

        gold = tgt[:, 1:]
        input_tgt = tgt[:, :-1]
        input_timestamp = tgt_timestamp[:, :-1] 


        n_words = gold.data.ne(Constants.PAD).sum().float()
        n_total_words += n_words
        batch_num += tgt.size(0)

        optimizer.zero_grad()
        
        pred = model(input_tgt, input_timestamp)  ##16 * 438 , 12629
        # backward

        loss = loss_func(pred, gold.contiguous().view(-1))   ##gold is 16 * 438 
        pred = pred.max(1)[1]

        gold = gold.contiguous().view(-1)
        n_correct = pred.data.eq(gold.data)
        n_correct = n_correct.masked_select(gold.ne(Constants.PAD).data).sum().float()

        loss.backward()

        # update parameters
        optimizer.step()
        optimizer.update_learning_rate()

        # note keeping
        n_total_correct += n_correct
        total_loss += loss.item()

        if i % 10 == 0:
            print("Training batch ", i, " loss: ", loss.item(), " acc:", (n_correct.item()/len(pred)), "n_correct:",n_correct.item(),
                 "n_words: ", n_words.item())
   

    train_loss =  total_loss/n_total_words 
    train_accu =  n_total_correct/n_total_words
    

    scores_train.append(doit(train_data,"train_data"))
    scores_valid.append(doit(valid_data,"valid_data"))
    scores_test.append(doit(test_data,"test_data"))
    print('  - (Training)   loss: {loss: 8.5f}, accuracy: {accu:3.3f} %'.format(loss=train_loss, accu=100 * train_accu))



Training batch  0  loss:  9513.2470703125  acc: 0.0 n_correct: 0.0 n_words:  996.0
Training batch  10  loss:  4264.0576171875  acc: 0.0 n_correct: 0.0 n_words:  445.0
Training batch  20  loss:  4893.3984375  acc: 0.0 n_correct: 0.0 n_words:  511.0
Training batch  30  loss:  5566.83935546875  acc: 0.0 n_correct: 0.0 n_words:  582.0
Training batch  40  loss:  4065.058349609375  acc: 0.0005681818181818182 n_correct: 1.0 n_words:  430.0
Training batch  50  loss:  3646.0859375  acc: 0.002577319587628866 n_correct: 4.0 n_words:  384.0
Training batch  60  loss:  3889.54296875  acc: 0.0043859649122807015 n_correct: 8.0 n_words:  410.0
Training batch  70  loss:  4148.44140625  acc: 0.0037162162162162164 n_correct: 11.0 n_words:  442.0
Training batch  80  loss:  4634.73388671875  acc: 0.004310344827586207 n_correct: 16.0 n_words:  498.0
Training batch  90  loss:  3076.833984375  acc: 0.005780346820809248 n_correct: 16.0 n_words:  336.0
Training batch  100  loss:  8491.0849609375  acc: 0.00552486

NameError: name 'scores_train' is not defined

In [12]:
doit(test_data,"test_data")

test_data batch  0
hits@10 0.058331713007972005
map@10 0.047290353045748725
hits@50 0.0944418210605261
map@50 0.04885299331294
hits@100 0.13610733035193465
map@100 0.04940690460038843


{'hits@10': 0.058331713007972005,
 'map@10': 0.047290353045748725,
 'hits@50': 0.0944418210605261,
 'map@50': 0.04885299331294,
 'hits@100': 0.13610733035193465,
 'map@100': 0.04940690460038843}

In [13]:
scores_train2 = pd.DataFrame(scores_train)
scores_valid2 = pd.DataFrame(scores_valid)
scores_test2 = pd.DataFrame(scores_test)

NameError: name 'scores_train' is not defined

In [14]:
scores_test2

NameError: name 'scores_test2' is not defined

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [15]:

for metr in scores_train2.columns:
    x_train = scores_train2[metr]
    x_valid = scores_valid2[metr]
    x_test = scores_test2[metr]
    plt.plot(x_train)
    plt.plot(x_valid)
    plt.plot(x_test)
    
    plt.title(metr)

    plt.legend(('Training set','Validation set','Test set'))

    plt.show()

NameError: name 'scores_train2' is not defined